# Project Setup
* This notebok describes how to create and configure a project
* This is the same as creating a new project in the editor and going through all of the steps.

* When a user creates a project with client.create_project() the project is not ready for labeling.
    * An ontology must be set
    * datasets must be attached

In [ ]:
!pip install labelbox

In [ ]:
from labelbox import Client, Project, Tool, OntologyBuilder

In [ ]:
# Add your api key
API_KEY = None
ENDPOINT = "https://api.labelbox.com/graphql"
client = Client(api_key=API_KEY, endpoint=ENDPOINT)

### Create Dataset
* Create dataset and attach data
* More details on attaching data can be found [here](https://github.com/Labelbox/labelbox-python/blob/develop/examples/basics/data_rows.ipynb)

In [ ]:
dataset = client.create_dataset(name="my_new_dataset")
# Add data_rows since this is a new dataset (see basics/data_rows.ipynb for more information on this)
test_img_url = "https://raw.githubusercontent.com/Labelbox/labelbox-python/develop/examples/assets/2560px-Kitano_Street_Kobe01s5s4110.jpg"
dataset.create_data_row(row_data=test_img_url)

### Create Or Select an Ontology
* Optionally create an ontology or select from an existing one.
* More details on ontology management can be found [here](https://github.com/Labelbox/labelbox-python/blob/develop/examples/basics/ontologies.ipynb)
   

In [ ]:
# Create
ontology_builder = OntologyBuilder(
    tools=[Tool(name="cat", tool=Tool.Type.BBOX)])
ontology = client.create_ontology("project-setup-demo-ontology", ontology_builder.asdict())
# Select existing ontology
#ontology = client.get_ontology("<ontology_id>")
#ontology = existing_project.ontology()


### Create Project and Setup the Editor
* Setting up a project will add an ontology and will enable labeling to begin
* Attaching dataset(s) will add all data_rows belonging to the dataset to the queue.

In [ ]:
project = client.create_project(name="my_new_project")
project.setup_editor(ontology)
project.datasets.connect(dataset)

### Review

In [ ]:
# Note setup_complete will be None if it fails.
print(project.setup_complete)
print(project.ontology())
print([ds.name for ds in project.datasets()])

In [ ]:
print(f"https://app.labelbox.com/projects/{project.uid}")